In [2]:
!pip3 install --no-deps cell-data-loader
!pip2 install tkinter


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from cell_data_loader import CellDataloader
import torch,os,glob
from torchvision.models import resnet50 #, ResNet50_Weights
import tkinter as tk
import tkinter.filedialog


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cell_data_loader/util.py:19: UserWarning: Working Cellpose not installed -- cell segmentation not possible
  warnings.warn("Working Cellpose not installed -- cell segmentation not"+\


In [2]:
### CHANGE THESE LINES
wd = '/Users/mleming/Desktop/CellDataLoader_playground'
local_folder_name = os.path.join(wd,'dropbox_downloads')
use_tkinter = True
root = tk.Tk()
if use_tkinter:
    print("Select your working directory to save the model")
    wd = tk.filedialog.askdirectory(parent=root,title="Please select the directory to save the model")
    print("Select the directory with the cell data")
    local_folder_name = tk.filedialog.askdirectory(title="Please select the directory with cell data")
print(f"Working directory to save model: {wd}")
print(f"Location of data: {local_folder_name}")


Working directory to save model: /Users/mleming/Desktop/CellDataLoader_playground
Location of data: /Users/mleming/Desktop/CellDataLoader_playground/dropbox_downloads


In [4]:
os.makedirs(wd, exist_ok = True)
os.makedirs(local_folder_name, exist_ok = True)

# Checkpoints
model_folder = os.path.join(wd,'checkpoints')
os.makedirs(model_folder,exist_ok=True)
model_file = os.path.join(model_folder,'torch_model.pt')

# Training data folders
imfolder1_train = glob.glob(os.path.join(local_folder_name,'*','Blurry'))
imfolder2_train =  glob.glob(os.path.join(local_folder_name,'*','Clear'))

imfolder1_test = os.path.join(local_folder_name,'3368914_4_non_tumor')
imfolder2_test = os.path.join(local_folder_name,'4173633_5')


In [7]:
def train_torch(model,
                dataloader,
                model_file = None,
                epochs = 50,
                gpu_ids = None,
                verbose = True):

    if gpu_ids is not None:
        model.to(gpu_ids)

    # Train

    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    if verbose: print("Beginning training")
    for epoch in range(epochs):
        loss_total,count = 0,0
        for image,y in dataloader_train:
            y_pred = model(image)
            y_pred = y_pred[:,:y.size()[1]]
            
            loss = loss_fn(y_pred, torch.argmax(y,1))
            loss_total += float(loss)
            count += 1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if model_file is not None:
            torch.save(model,model_file)
        if verbose:
            print(
                "Epoch {epoch:d}/{epochs:d}: loss: {loss:.5f}".format(
                    epoch=epoch,epochs=epochs,loss=loss_total/count)
            )

def test_torch(model,
               dataloader,
               gpu_ids = None,
               verbose=True):
    
    model.eval()
    total_images = 0
    sum_accuracy = 0
    for image,y in dataloader:
        total_images += image.size()[0]
        y_pred = model(image)
        y_pred = y_pred[:,:y.size()[1]]
        sum_accuracy += torch.sum(torch.argmax(y_pred,axis=1) == \
            torch.argmax(y,axis=1))
    accuracy = sum_accuracy / total_images
    if verbose: print("Final accuracy: %.4f" % accuracy)


In [8]:
# Get a pretrained model from torchvision
if os.path.isfile(model_file):
    print("Loading %s" % model_file)
    model = torch.load(model_file)
else:
    model = resnet50(pretrained=True)

dataloader_train = CellDataloader(local_folder_name,label_regex=["Blurry","Clear"],
    dtype = "torch",
    verbose = False,
    batch_size = 64,
    gpu_ids = None)

dataloader_test = 

Loading /Users/mleming/Desktop/CellDataLoader_playground/checkpoints/torch_model.pt


In [44]:
train_torch(model,dataloader_train)

Beginning training
torch.Size([64, 3, 64, 64])
tensor([0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638, 0.0638,
        0.0638])
torch.Size([64, 3, 64, 64])
tensor([0.8844, 0.8844, 0.8844, 0.8767, 0.8844, 0.8844, 0.8765, 0.8768, 0.8844,
        0.8874, 0.8844, 0.8844, 0.8844, 0.8844, 0.8844, 0.8868, 0.8844, 0.8844,
        0.8844, 0.8844, 0.8844, 0.8844, 0.8844, 0.8844, 0.8880, 0.8844, 0.8844,
        0.8869, 0.8844, 0.8844, 0.8844, 0.8844, 0.8844, 0.8844, 0.8873, 0.8844,
        0.8844, 0.8844, 0.88

KeyboardInterrupt: 